Stub to test how to connect to Microsoft Graph for accessing MS Outlook (new) on Windows 11

To use this version:
1. Make sure Windows 11 Outlook (new) client is running and the AOL account is connected
2. Connect via Microsoft Graph
3. Go through all the mail object in the "Bulk Mail" folder
   1. print the from address and subject

Would you like me to:
3. Add email volume reporting?
4. Create a summary report of processed emails?

In [1]:

# Todo
# Convert from using win32com to using o365
# from O365 import Account

# credentials = ('your_client_id', 'your_client_secret')
# account = Account(credentials)
# if account.authenticate(scopes=['basic', 'message_all']):
#     mailbox = account.mailbox()
#     m = mailbox.new_message()
#     m.to.add('[email protected]')
#     m.subject = 'Subject'
#     m.body = 'Body'
#     m.send()
#   see https://developer.microsoft.com/en-us/graph/graph-explorer
#   see authentication script below
# from msal import PublicClientApplication

# CLIENT_ID = "your_client_id"  # From your app registration in the new tenant or an existing one you have access to
# SCOPES = ["User.Read"]  # Start with basic permissions, then add more as needed

# app = PublicClientApplication(
#     client_id=CLIENT_ID,
#     authority="https://login.microsoftonline.com/common"  # Use 'common' for personal accounts
# )

# # Interactive authentication - this will open a browser for login
# result = app.acquire_token_interactive(scopes=SCOPES)

# if "access_token" in result:
#     print(result["access_token"])
# else:
#     print(result.get("error"))
#     print(result.get("error_description"))
#--------------------------------------------
# Add "easy to add to Outlook Rules"
#  - Track all the "No conditions or phishing indicators found" as you go by Outlook Rule:
#     - so that you can write a summary at the end, keep a record for each of From:, Subject:, Body:, Header:
#  - Then list summary at the end:
#    - make it easy to copy/paste into Outlook Rules, one rule at a time.
#   Body - then one line per with .<domain>. and /<domain>.
#   From - one line per with @<domain>.
#   Subject/From - one line with From: trimmed to "@<domain>.", Subject: <subject>

In [10]:
#Imports for python base packages
import re
import time
import logging
import sys
import os
import json
import webbrowser

#Imports for packages that need to be installed
import win32com.client
import IPython
from msal import PublicClientApplication
from O365 import Account
import requests

In [11]:
DEBUG = True
INFO = False if DEBUG else True #If not debugging, then INFO level logging
DEBUG_EMAILS_TO_PROCESS = 100
EMAIL_ADDRESS = "kimmeyharold@aol.com"
EMAIL_FOLDER_NAME = "Bulk Mail"
WIN32_CLIENT_DISPATCH = "Outlook.Application"
OUTLOOK_GETNAMESPACE = "MAPI"
OUTLOOK_SECURITY_LOG = "outlook_security.log"

DAYS_BACK_DEFAULT = 365 #default number of days to go back in the calendar

CLIENT_ID = "ae895e3a-2376-4585-8d39-890983803f66"  # Replace with your application client ID for MyOutlookManager
# Graph API endpoint for mail
GRAPH_API_ENDPOINT = "https://graph.microsoft.com/v1.0/me/messages"
TOKEN_FILE = "token.json"

In [ ]:
def save_token(token):
    """Save the token to a file."""
    with open(TOKEN_FILE, "w") as f:
        json.dump(token, f)

def load_token():
    """Load the token from a file."""
    if os.path.exists(TOKEN_FILE):
        with open(TOKEN_FILE, "r") as f:
            return json.load(f)
    return None

def get_access_token():
    """Authenticate and get an access token using device code flow or refresh token."""
    authority = "https://login.microsoftonline.com/common"
    app = PublicClientApplication(CLIENT_ID, authority=authority)

    # Try to load the token from file
    token = load_token()
    if token:
        # Try to use the refresh token
        result = app.acquire_token_by_refresh_token(token["refresh_token"], scopes=["Mail.Read", "Mail.Send"])
        if "access_token" in result:
            save_token(result)
            return result["access_token"]

    # If no valid token, use device code flow
    while True:
        flow = app.initiate_device_flow(scopes=["Mail.ReadWrite.Shared", "Mail.Send"])
        if "user_code" not in flow:
            print("Failed to create device flow", flow)
            return None

        print(f"To sign in, use a web browser to open {flow['verification_uri']} and enter the code: {flow['user_code']}")
        print("You have 15 minutes to complete the authentication process.")
        print(f"Token generation time: {time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime())}")

        result = app.acquire_token_by_device_flow(flow)
        print(f"Result: {result}")
        print(f"Token validation time: {time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime())}")

        if "access_token" in result:
            save_token(result)
            return result["access_token"]
        elif result.get("error") == "authorization_pending":
            print("Authorization pending. Please complete the authentication process.")
        elif result.get("error") == "code_expired":
            print("The code you entered has expired. Get a new code from the device you're trying to sign in to and try again.")
        else:
            print("Failed to acquire token:", result.get("error_description"))
            return None

def fetch_emails(access_token):
    """Fetch emails from the Outlook mailbox."""
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.get(GRAPH_API_ENDPOINT, headers=headers)

    if response.status_code == 200:
        emails = response.json().get("value", [])
        for i, email in enumerate(emails, start=1):
            print(f"Email {i}:")
            print(f"Subject: {email['subject']}")
            print(f"From: {email['from']['emailAddress']['address']}")
            print("---")
    else:
        print("Failed to fetch emails:", response.status_code, response.text)


token = get_access_token()
if token:
    fetch_emails(token)


To sign in, use a web browser to open https://microsoft.com/devicelogin and enter the code: FN37URN49
You have 15 minutes to complete the authentication process.
Token generation time: 2025-01-01 02:09:48


In [ ]:
import webbrowser

# Replace with your values
client_secret = "YOUR_CLIENT_SECRET"
redirect_uri = "YOUR_REDIRECT_URI"
scopes = ["openid", "profile", "offline_access", "Mail.Read"]  # Adjust as needed

# 1. User signs in and grants consent
authorize_url = (
    "https://login.microsoftonline.com/common/oauth2/v2.0/authorize"
    f"?client_id={CLIENT_ID}"
    f"&response_type=code"
    f"&redirect_uri={redirect_uri}"
    f"&response_mode=query"
    f"&scope={'%20'.join(scopes)}"
    "&state=12345"
)

print("Opening browser to authenticate...")
webbrowser.open(authorize_url)
print("After you sign in, you will be redirected to the redirect URI with a code.")

# The user will have to copy the 'code' from the redirect URI manually.
# In a local test scenario, set up a local server or instruct the user to paste the code.
authorization_code = input("Enter the authorization code: ")

# 2. Exchange the authorization code for an access token
token_url = "https://login.microsoftonline.com/common/oauth2/v2.0/token"
data = {
    "client_id": client_id,
    "client_secret": client_secret,
    "code": authorization_code,
    "grant_type": "authorization_code",
    "redirect_uri": redirect_uri,
}
token_response = requests.post(token_url, data=data)
if token_response.status_code != 200:
    print("Error obtaining token:", token_response.text)
    exit(1)

tokens = token_response.json()
access_token = tokens.get("access_token")

print("Access token obtained successfully.")

# 3. Use the access token to call the Microsoft Graph API and read the user's mailbox
graph_url = "https://graph.microsoft.com/v1.0/me/mailFolders/Inbox/messages?$top=1"
headers = {
    "Authorization": f"Bearer {access_token}"
}

graph_response = requests.get(graph_url, headers=headers)
if graph_response.status_code == 200:
    result = graph_response.json()
    print("Successfully accessed the inbox!")
    print("Sample message data:", result)
else:
    print("Error accessing the inbox:", graph_response.text)


In [14]:
class OutlookSecurityAgent:
    def __init__(self, email_address=EMAIL_ADDRESS, folder_name=EMAIL_FOLDER_NAME, debug_mode=DEBUG):
        """
        Initialize the Outlook Security Agent with specific account and folder

        Args:
            email_address: Email address of the account to process
            folder_name: Name of the folder to process
            debug_mode: If True, run in simulation mode with verbose output
        """
        self.debug_mode = debug_mode
        self.outlook = win32com.client.Dispatch(WIN32_CLIENT_DISPATCH)
        self.namespace = self.outlook.GetNamespace(OUTLOOK_GETNAMESPACE)

        # Configure logging
        log_format = '%(asctime)s - %(levelname)s - %(message)s'
        logging.basicConfig(
            level=logging.DEBUG if debug_mode else logging.INFO,
            format=log_format,
            handlers=[
                logging.FileHandler(OUTLOOK_SECURITY_LOG),
                logging.StreamHandler(sys.stdout)  # Also print to console
            ]
        )

        self.debug_print(f"Initializing agent for {email_address}, folder: {folder_name}")
        self.debug_print(f"Debug mode: {debug_mode}")

        # Get the specific account's folder
        self.target_folder = self._get_account_folder(email_address, folder_name)
        if not self.target_folder:
            raise ValueError(f"Could not find folder '{folder_name}' in account '{email_address}'")

        self.rules = []
        self.rule_to_category = {
            "SpamAutoDeleteBody":           "SpamBody",
            "SpamAutoDeleteBody-imgur.com": "SpamImgur",
            "SpamAutoDeleteFrom":           "SpamHeader",
            "SpamAutoDeleteHeader":         "SpamHeader",
            "SpamAutoDeleteSubject":        "SpamSubject"
        }

    def debug_print(self, message):
        try:
            sanitized_message = self._sanitize_string(message)
            logging.debug(sanitized_message)
        except UnicodeEncodeError:
            logging.debug(sanitized_message.encode('utf-8', 'replace').decode('utf-8'))

    def _sanitize_string(self, s):
        """Sanitize string to replace non-ASCII characters"""
        return re.sub(r'[^\x00-\x7F]+', '', s)

    def _get_account_folder(self, email_address, folder_name):
        """Get a specific folder from a specific email account"""
        self.debug_print(f"Searching for folder: {folder_name} in account: {email_address}")

        try:
            # Loop through accounts to find the matching one
            for account in self.outlook.Session.Accounts:
                self.debug_print(f"Checking account: {account.SmtpAddress}")

                if account.SmtpAddress.lower() == email_address.lower():
                    self.debug_print(f"Found matching account: {account.SmtpAddress}")

                    # Get the root folder for this account
                    root_folder = self.namespace.Folders(account.DeliveryStore.DisplayName)
                    self.debug_print(f"Accessed root folder: {root_folder.Name}")

                    # Search for the target folder
                    try:
                        # Try direct access first
                        target_folder = root_folder.Folders[folder_name]
                        self.debug_print(f"Found target folder directly: {folder_name}")
                        return target_folder
                    except Exception:
                        self.debug_print(f"Folder not found directly, searching recursively...")
                        return self._find_folder_recursive(root_folder, folder_name)

            self.debug_print(f"Account not found: {email_address}")
            return None

        except Exception as e:
            self.debug_print(f"Error finding account folder: {str(e)}")
            return None

    def get_outlook_rules(self):
        """
        Convert Outlook rules to JSON format with comprehensive error checking.
        Returns a list of rule dictionaries with all available properties.
        """
        self.debug_print("Converting Outlook rules to JSON format...")
        rules_json = []

        try:
            # NOTE: GetRules() is not returning several of the actions:
            #   - Mark as Read
            #   - Clear the Message Flag
            #   - Stop Processing More Rules
            #   Also, the "AssignToCategory" is not returning the category name
            outlook_rules = self.outlook.Session.DefaultStore.GetRules()
            self.debug_print(f"Processing {len(outlook_rules)} rules...")

                    # Process Actions
                    if hasattr(rule, "Actions") and rule.Actions:
                    if "SpamAutoDelete" not in rule.Name:
                        continue

                    # Process Exceptions
                    if hasattr(rule, "Exceptions") and rule.Exceptions:
                        exceptions = rule.Exceptions
                        "name": rule.Name if hasattr(rule, "Name") else "Unnamed Rule",
                        "enabled": bool(rule.Enabled) if hasattr(rule, "Enabled") else False,
                        "isLocal": bool(rule.IsLocalRule) if hasattr(rule, "IsLocalRule") else False,
                        "executionOrder": rule.ExecutionOrder if hasattr(rule, "ExecutionOrder") else 0,
                        "conditions": {},
                        "actions": {},
                        "exceptions": {}
                    }

                    # Process Conditions
                    if hasattr(rule, "Conditions") and rule.Conditions:
                        conditions = rule.Conditions
                        rule_dict["conditions"] = self._process_conditions(conditions, False)

                    # Process Actions
                    if hasattr(rule, "Actions") and rule.Actions:
                        actions = rule.Actions
                        rule_dict["actions"] = self._process_actions(actions)

                    # Process Exceptions
                    if hasattr(rule, "Exceptions") and rule.Exceptions:
                        exceptions = rule.Exceptions
                        rule_dict["exceptions"] = self._process_conditions(exceptions, True)  # Exceptions use same format as conditions

                    rules_json.append(rule_dict)
                    self.debug_print(f"Successfully processed rule: {rule_dict['name']}")

                except Exception as e:
                    self.debug_print(f"Error processing rule {getattr(rule, 'Name', 'Unknown')}: {str(e)}")
                    # Add error information to the rule
                    rules_json.append({
                        "name": getattr(rule, "Name", "Unknown Rule"),
                        "error": str(e),
                        "processed": False
                    })

            # print (json.dumps(rules_json, indent=2, default=str))
            return json.dumps(rules_json, indent=2, default=str)

        except Exception as e:
            self.debug_print(f"Error accessing Outlook rules: {str(e)}")
            return json.dumps({"error": str(e)})

    def _process_conditions(self, conditions_obj, is_exception):
        """Helper method to process rule conditions or exceptions"""
        conditions = {}

        try:
            # From addresses
            if hasattr(conditions_obj, "From") and conditions_obj.From:
                try:
                    conditions["from"] = [
                        {
                            "address": recipient.Address if hasattr(recipient, "Address") else None,
                            "name": recipient.Name if hasattr(recipient, "Name") else None
                        }
                        for recipient in conditions_obj.From.Recipients
                    ]
                    # Print the contents of conditions["from"]
                    if is_exception:
                        self.debug_print(f"Exception conditions['from']: {conditions['from']}")
                    else:
                        self.debug_print(f"Conditions['from']: {conditions['from']}")

                except Exception as e:
                    self.debug_print(f"Error processing From condition: {str(e)}")
                    conditions["from"] = []

            # Subject keywords
            if hasattr(conditions_obj, "Subject") and conditions_obj.Subject:
                try:
                    if is_exception:
                        self.debug_print(f"Exception conditions_obj.Subject.Text: {conditions_obj.Subject.Text}")
                    else:
                        self.debug_print(f"Conditions_obj.Subject.Text: {conditions_obj.Subject.Text}")

                    if hasattr(conditions_obj.Body, "Text"):
                        if isinstance(conditions_obj.Body.Text, str):
                            subject_text = conditions_obj.Subject.Text
                        elif isinstance(conditions_obj.Subject.Text, tuple):
                            subject_text = "; ".join(conditions_obj.Subject.Text)
                        else:
                            subject_text = ""
                    else:
                        subject_text = ""
                    conditions["subject"] = [kw.strip() for kw in subject_text.split(";") if kw.strip()]
                except Exception as e:
                    self.debug_print(f"Error processing Subject condition: {str(e)}")
                    conditions["subject"] = []

            # Body keywords
            if hasattr(conditions_obj, "Body") and conditions_obj.Body:
                try:
                    if is_exception:
                        self.debug_print(f"Exception conditions_obj.Body.Text: {conditions_obj.Body.Text}")
                    else:
                        self.debug_print(f"Conditions_obj.Body.Text: {conditions_obj.Body.Text}")


                    if hasattr(conditions_obj.MessageHeader, "Text"):
                            body_text = conditions_obj.Body.Text
                        elif isinstance(conditions_obj.Body.Text, tuple):
                            body_text = "; ".join(conditions_obj.Body.Text)
                            # self.debug_print(f"body_text: {body_text}")
                        else:
                            body_text = ""
                    else:
                        body_text = ""
                    conditions["header"] = [kw.strip() for kw in header_text.split(";") if kw.strip()]
                except Exception as e:
                    self.debug_print(f"Error processing Body condition: {str(e)}")
                    conditions["body"] = []

            # Header keywords
            if hasattr(conditions_obj, "MessageHeader") and conditions_obj.MessageHeader:
                try:
                    if is_exception:
                        self.debug_print(f"Exception conditions_obj.MessageHeader.Text: {conditions_obj.MessageHeader.Text}")
                    else:
                        self.debug_print(f"Conditions_obj.MessageHeader.Text: {conditions_obj.MessageHeader.Text}")

                    if hasattr(conditions_obj.MessageHeader, "Text"):
                        if isinstance(conditions_obj.MessageHeader.Text, str):
                            header_text = conditions_obj.MessageHeader.Text
                        elif isinstance(conditions_obj.MessageHeader.Text, tuple):
                            header_text = "; ".join(conditions_obj.MessageHeader.Text)
                        else:
                            header_text = ""
                    else:
                        header_text = ""
                    conditions["header"] = [kw.strip() for kw in header_text.split(";") if kw.strip()]
                except Exception as e:
                    self.debug_print(f"Error processing Header condition: {str(e)}")
                    conditions["header"] = []

            # Attachment condition
            if hasattr(conditions_obj, "Attachment"):
                if is_exception:
                    self.debug_print(f"Exception conditions_obj.Attachment: {bool(conditions_obj.Attachment)}")
                else:
                    self.debug_print(f"Conditions_obj.Attachment: {bool(conditions_obj.Attachment)}")

                conditions["has_attachments"] = bool(conditions_obj.Attachment)

        except Exception as e:
            self.debug_print(f"Error processing conditions: {str(e)}")
            conditions["error"] = str(e)

        return conditions

    def _process_actions(self, actions_obj):
        """Helper method to process rule actions"""
        actions = {}

        try:
            # Move to Folder
            if hasattr(actions_obj, "MoveToFolder") and actions_obj.MoveToFolder:
                try:
                    actions["move_to_folder"] = {
                        "folder_path": actions_obj.MoveToFolder.FolderPath if hasattr(actions_obj.MoveToFolder, "FolderPath") else None,
                        "folder_name": actions_obj.MoveToFolder.Name if hasattr(actions_obj.MoveToFolder, "Name") else None
                    }
                except Exception as e:
                    self.debug_print(f"Error processing MoveToFolder action: {str(e)}")

            # Copy to Folder
            if hasattr(actions_obj, "CopyToFolder") and actions_obj.CopyToFolder:
                try:
                    actions["copy_to_folder"] = {
                        "folder_path": actions_obj.CopyToFolder.FolderPath if hasattr(actions_obj.CopyToFolder, "FolderPath") else None,
                        "folder_name": actions_obj.CopyToFolder.Name if hasattr(actions_obj.CopyToFolder, "Name") else None
                    }
                except Exception as e:
                    self.debug_print(f"Error processing CopyToFolder action: {str(e)}")

            # Assign to Category
            if hasattr(actions_obj, "AssignToCategory") and actions_obj.AssignToCategory:
                try:
                    category_name = actions_obj.AssignToCategory.Category if hasattr(actions_obj.AssignToCategory, "Category") else None
                    self.debug_print(f"AssignToCategory action found, category_name: {category_name}")
                    actions["assign_to_category"] = {
                        "category_name": category_name
                    }
                except Exception as e:
                    self.debug_print(f"Error processing AssignToCategory action: {str(e)}")

            # Delete
            if hasattr(actions_obj, "Delete") and actions_obj.Delete:
                actions["delete"] = True

            # Stop processing more rules
            if hasattr(actions_obj, "StopProcessingMoreRules") and actions_obj.StopProcessingMoreRules:
                try:
                    self.debug_print("StopProcessingMoreRules action found")
                    actions["stop_processing_more_rules"] = True
                except Exception as e:
                    self.debug_print(f"Error processing StopProcessingMoreRules action: {str(e)}")

            # Mark as Read
            if hasattr(actions_obj, "MarkAsRead") and actions_obj.MarkAsRead:
                try:
                    self.debug_print("MarkAsRead action found")
                    actions["mark_as_read"] = True
                except Exception as e:
                    self.debug_print(f"Error processing MarkAsRead action: {str(e)}")

            # Clear the Message Flag
            if hasattr(actions_obj, "ClearFlag") and actions_obj.ClearFlag:
                try:
                    self.debug_print("ClearFlag action found")
                    actions["clear_flag"] = True
                except Exception as e:
                    self.debug_print(f"Error processing ClearFlag action: {str(e)}")

            # Forward
            if hasattr(actions_obj, "Forward") and actions_obj.Forward:
                try:
                    actions["forward"] = [
                        {
                            "address": recipient.Address if hasattr(recipient, "Address") else None,
                            "name": recipient.Name if hasattr(recipient, "Name") else None
                        }
                        for recipient in actions_obj.Forward.Recipients
                    ]
                except Exception as e:
                    self.debug_print(f"Error processing Forward action: {str(e)}")
                    actions["forward"] = []

            # Redirect
            if hasattr(actions_obj, "Redirect") and actions_obj.Redirect:
                try:
                    actions["redirect"] = [
                        {
                            "address": recipient.Address if hasattr(recipient, "Address") else None,
                            "name": recipient.Name if hasattr(recipient, "Name") else None
                        }
                        for recipient in actions_obj.Redirect.Recipients
                    ]
                except Exception as e:
                    self.debug_print(f"Error processing Redirect action: {str(e)}")
                    actions["redirect"] = []

            # Reply
            if hasattr(actions_obj, "Reply") and actions_obj.Reply:
                try:
                    actions["reply"] = {
                        "template": actions_obj.Reply.Template if hasattr(actions_obj.Reply, "Template") else None
                    }
                except Exception as e:
                    self.debug_print(f"Error processing Reply action: {str(e)}")

            # Play Sound
            if hasattr(actions_obj, "PlaySound") and actions_obj.PlaySound:
                try:
                    actions["play_sound"] = {
                        "sound_file": actions_obj.PlaySound.SoundFile if hasattr(actions_obj.PlaySound, "SoundFile") else None
                    }
                except Exception as e:
                    self.debug_print(f"Error processing PlaySound action: {str(e)}")

            # Display Desktop Alert
            if hasattr(actions_obj, "DisplayDesktopAlert") and actions_obj.DisplayDesktopAlert:
                actions["display_desktop_alert"] = True

            # Set Importance
            if hasattr(actions_obj, "SetImportance") and actions_obj.SetImportance:
                try:
                    actions["set_importance"] = {
                        "importance_level": actions_obj.SetImportance.ImportanceLevel if hasattr(actions_obj.SetImportance, "ImportanceLevel") else None
                    }
                except Exception as e:
                    self.debug_print(f"Error processing SetImportance action: {str(e)}")

            # Set Sensitivity
            if hasattr(actions_obj, "SetSensitivity") and actions_obj.SetSensitivity:
                try:
                    actions["set_sensitivity"] = {
                        "sensitivity_level": actions_obj.SetSensitivity.SensitivityLevel if hasattr(actions_obj.SetSensitivity, "SensitivityLevel") else None
                    }
                except Exception as e:
                    self.debug_print(f"Error processing SetSensitivity action: {str(e)}")

            # Print
            if hasattr(actions_obj, "Print") and actions_obj.Print:
                actions["print"] = True

            # Run Script
            if hasattr(actions_obj, "RunScript") and actions_obj.RunScript:
                try:
                    actions["run_script"] = {
                        "script_path": actions_obj.RunScript.ScriptPath if hasattr(actions_obj.RunScript, "ScriptPath") else None
                    }
                except Exception as e:
                    self.debug_print(f"Error processing RunScript action: {str(e)}")

            # Start Application
            if hasattr(actions_obj, "StartApplication") and actions_obj.StartApplication:
                try:
                    actions["start_application"] = {
                        "application_path": actions_obj.StartApplication.ApplicationPath if hasattr(actions_obj.StartApplication, "ApplicationPath") else None
                    }
                except Exception as e:
                    self.debug_print(f"Error processing StartApplication action: {str(e)}")

            # Mark as Task
            if hasattr(actions_obj, "MarkAsTask") and actions_obj.MarkAsTask:
                try:
                    actions["mark_as_task"] = {
                        "task_due_date": actions_obj.MarkAsTask.TaskDueDate if hasattr(actions_obj.MarkAsTask, "TaskDueDate") else None
                    }
                except Exception as e:
                    self.debug_print(f"Error processing MarkAsTask action: {str(e)}")

        except Exception as e:
            self.debug_print(f"Error processing actions: {str(e)}")

        return actions

    def check_phishing_indicators(self, email): #*** not currently called
        """Check for phishing indicators in an email"""
        indicators = []
                href_pattern = r'href=[\'"]?([^\'" >]+)'
        try:
            # self.debug_print(f"From: {email.SenderEmailAddress}")
            # self.debug_print(f"Subject: {email.Subject}")

            # Check sender mismatch
            sender = email.SenderEmailAddress.lower()
            display_name = email.SenderName.lower()
            if '@' in display_name and display_name != sender:
                self.debug_print(f"Found sender mismatch: {display_name} vs {sender}")
                indicators.append("Sender name/email mismatch")

            # Check urgent language
            urgent_words = ['urgent', 'immediate', 'action required', 'account suspended']
            found_urgent = [word for word in urgent_words if word in email.Subject.lower()]
            if found_urgent:
                self.debug_print(f"Found urgent language: {found_urgent}")
                indicators.append("Urgent language in subject")

            # Check URLs
            if email.HTMLBody:
                href_pattern = r'href=[\'"]?([^\'" >]+)'
                urls = re.findall(href_pattern, email.HTMLBody)
                for url in urls:
                    if 'http' in url.lower():
                        if url.lower() not in email.HTMLBody.lower():
                            self.debug_print(f"Found mismatched URL: {url}")
                            indicators.append("Mismatched URL display text")
                            break

            # Check sensitive words
            sensitive_words = ['password', 'login', 'credential', 'verify account']
            found_sensitive = [word for word in sensitive_words if word in email.Body.lower()]
            if found_sensitive:
                self.debug_print(f"Found sensitive words: {found_sensitive}")
                indicators.append("Requests for sensitive information")

            # Check Outlook rules
            for rule in self.rules:
                self.debug_print(f"\nChecking rule: {rule['name']}")
                conditions = rule['conditions']
                match = True

                for condition_type, condition_value in conditions.items():
                    self.debug_print(f"  Checking condition: {condition_type}")
                    if condition_type == 'from_addresses':
                        if not any(addr.lower() in sender for addr in condition_value):
                            match = False
                            self.debug_print("    Sender address did not match")
                            break
                    elif condition_type == 'subject_keywords':
                        if not any(keyword.lower() in email.Subject.lower() for keyword in condition_value):
                            match = False
                            self.debug_print("    Subject keywords did not match")
                            break
                    elif condition_type == 'body_keywords':
                        if not any(keyword.lower() in email.Body.lower() for keyword in condition_value):
                            match = False
                            self.debug_print("    Body keywords did not match")
                            break
                    elif condition_type == 'has_attachments':
                        if bool(email.Attachments.Count > 0) != condition_value:
                            match = False
                            self.debug_print("    Attachment condition did not match")
                            break

                if match:
                    self.debug_print(f"  Rule matched: {rule['name']}")
                    indicators.append(f"Matched Outlook rule: {rule['name']}")

        except Exception as e:
            self.debug_print(f"Error checking indicators: {str(e)}")

        return indicators

    def process_emails(self, rules_json, days_back=DAYS_BACK_DEFAULT):
        """Process emails based on the rules in the rules_json object"""
        self.debug_print(f"\n\nStarting email processing (Simulation Mode)")
        self.debug_print(f"Target folder: {self.target_folder.Name}")
        self.debug_print(f"Processing emails from last {days_back} days")

        try:
            # Parse the rules from the JSON object
            rules = json.loads(rules_json)

            # Get recent emails from the target folder
            restriction = "[ReceivedTime] >= '" + \
                (datetime.now() - timedelta(days=days_back)).strftime('%m/%d/%Y') + "'"
            emails = self.target_folder.Items.Restrict(restriction)
            emails.Sort("[ReceivedTime]", Descending=True)
            self.debug_print(f"Total emails found: {emails.Count}")

            processed_count = 0
            flagged_count = 0

            self.debug_print("Beginning email analysis:")

            # Create a list of emails to process (done because if deleting emails in "email in emails") it will skip emails
            emails_to_process = [email for email in emails]

            for email in emails_to_process:
                try:
                    processed_count += 1
                    email_deleted = False
                    if (DEBUG) and (processed_count > DEBUG_EMAILS_TO_PROCESS):
                        self.debug_print(f"Debug mode: Stopping after {DEBUG_EMAILS_TO_PROCESS} emails")
                        return
                    email_header = email.PropertyAccessor.GetProperty("http://schemas.microsoft.com/mapi/proptag/0x007D001E")
                    self.debug_print(f"\n\nEmail {processed_count}:")
                    self.debug_print(f"Subject: {self._sanitize_string(email.Subject)}")
                    self.debug_print(f"From: {self._sanitize_string(email.SenderEmailAddress)}")
                    self.debug_print(f"Received: {email.ReceivedTime}")
                    # for header in email_header.splitlines():
                    #     self.debug_print(f"Header: {header}")
                    # self.debug_print(f"Body: {self._sanitize_string(email.Body)}")
                    # self.debug_print(f"Attachments: {email.Attachments.Count}")


                    # Sort rules to ensure delete actions are processed last
                    rules.sort(key=lambda rule: rule['actions'].get('delete', False))

                    for rule in rules:
                        if email_deleted:
                            continue  # Go to the next email if one rule deletes the current email
                        conditions = rule['conditions']
                        exceptions = rule['exceptions']
                        # print(rule, conditions)
                        match = False

                        #*** may have to change all conditions[] to lower case
                        #*** will likely have to add additional printing to see what is going on
                        # Check 'from' addresses
                        if 'from' in conditions:
                            from_addresses = [addr['address'].lower() for addr in conditions['from']]
                            if any(addr in email.SenderEmailAddress.lower() for addr in from_addresses):
                                match = True
                                matched_keyword = next((addr['address'] for addr in conditions['from'] if addr['address'].lower() in email.SenderEmailAddress.lower()), None)
                                self.debug_print(f"Matched keyword in from address: {matched_keyword}")
                                self.debug_print(f"From: {self._sanitize_string(email.SenderEmailAddress)}")

                        # Check subject keywords
                        if 'subject' in conditions:
                            if any(keyword.lower() in email.Subject.lower() for keyword in conditions['subject']):
                                match = True
                                matched_keyword = next((keyword for keyword in conditions['subject'] if keyword.lower() in email.Subject.lower()), None)
                                self.debug_print(f"Matched keyword in subject: {matched_keyword}")
                                self.debug_print(f"Subject: {self._sanitize_string(email.Subject)}")

                        # Check body keywords
                        if 'body' in conditions:
                            if any(keyword.lower() in email.Body.lower() for keyword in conditions['body']):
                                match = True
                                matched_keyword = next((keyword for keyword in conditions['body'] if keyword.lower() in email.Body.lower()), None)
                                self.debug_print(f"Matched keyword in body: {matched_keyword}")
                                matched_lines = [line for line in email.Body.splitlines() if matched_keyword.lower() in line.lower()]
                                if matched_lines:
                                    self.debug_print(f"First line of body that matches the keyword: {matched_lines[0]}")
                                # below will print all the body lines that match if needed for debugging
                                # for line in email.Body.splitlines():
                                #     if any(keyword.lower() in line.lower() for keyword in conditions['body']):
                                #         self.debug_print(f"Body: {line}")

                        # Check header keywords
                        if 'header' in conditions:
                            if any(keyword.lower() in email_header for keyword in conditions['header']):
                                match = True
                                matched_keyword = next((keyword for keyword in conditions['header'] if keyword.lower() in email_header.lower()), None)
                                self.debug_print(f"Matched keyword in header: {matched_keyword}")
                                matched_lines = [line for line in email_header.splitlines() if matched_keyword.lower() in line.lower()]
                                if matched_lines:
                                    self.debug_print(f"First line of header that matches the keyword: {matched_lines[0]}")
                                # below will print all the body lines that match if needed for debugging
                                # for header in email_header.splitlines():
                                #     if any(keyword.lower() in header.lower() for keyword in conditions['header']):
                                #         self.debug_print(f"Header: {header}")

                        # Check for attachments - not using. could be added later - will need to be updated; will not work as-is
                        # if 'has_attachments' in conditions:
                        #     if bool(email.Attachments.Count > 0) != conditions['has_attachments']:
                        #         match = True

                    # Check exceptions
                        if 'from' in exceptions:
                            from_addresses = [addr['address'].lower() for addr in exceptions['from']]
                            if any(addr in email.SenderEmailAddress.lower() for addr in from_addresses):
                        # Check header keywords in exceptions
                        if 'header' in exceptions:
                            if any(keyword.lower() in email_header for keyword in exceptions['header']):
                                match = False
                                matched_keyword = next((keyword for keyword in exceptions['header'] if keyword.lower() in email_header.lower()), None)
                                self.debug_print(f"Exception matched keyword in header: {matched_keyword}")
                                for header in email_header.splitlines():
                                    self.debug_print(f"Header: {header}")

                        # # Check for attachments - not using. could be added later - will need to be updated; will not work as-is
                        # if 'has_attachments' in conditions:
                        #     if bool(email.Attachments.Count > 0) != conditions['has_attachments']:
                        #         match = False

                        if match:
                            self.debug_print(f"Email matches rule: {rule['name']}")
                            # Perform actions based on the rule
                            actions = rule['actions']
                            self.debug_print(f"Performing actions: {actions}")
                            if 'assign_to_category' in actions and actions['assign_to_category']['category_name']:
                                rule_name = rule['name']
                                category_name = self.rule_to_category.get(rule_name, actions['assign_to_category']['category_name'])
                                email.Categories = category_name
                                email.Save()
                                self.debug_print(f"Email assigned to category '{category_name}'")
                            if 'assign_to_category' in actions and actions['assign_to_category']['category_name']:
                                # ***category name is not being passed.  Will setup a table per email rule to assign it "semi-manually"
                                email.Categories = actions['assign_to_category']['category_name']
                                email.Save()
                                self.debug_print(f"Email assigned to category '{actions['assign_to_category']['category_name']}'")
                            if 'mark_as_read' in actions and actions['mark_as_read']:
                                # this flag is not being passed by outlook, so will never be set.  Keeping in case fixed in the future
                                email.UnRead = False
                                self.debug_print("Email marked as read")
                            if 'clear_flag' in actions and actions['clear_flag']:
                                # this flag is not being passed by outlook, so will never be set.  Keeping in case fixed in the future
                                email.Flag.Clear()
                                self.debug_print("Email flag cleared")
                            if 'set_importance' in actions and actions['set_importance']['importance_level']:
                                email.Importance = actions['set_importance']['importance_level']
                                email.Save()
                                self.debug_print(f"Email importance set to {actions['set_importance']['importance_level']}")
                            if 'set_sensitivity' in actions and actions['set_sensitivity']['sensitivity_level']:
                                email.Sensitivity = actions['set_sensitivity']['sensitivity_level']
                                email.Save()
                                self.debug_print(f"Email sensitivity set to {actions['set_sensitivity']['sensitivity_level']}")
                            if 'mark_as_task' in actions and actions['mark_as_task']['task_due_date']:
                                email.TaskDueDate = actions['mark_as_task']['task_due_date']
                                email.Save()
                                self.debug_print(f"Email marked as task with due date: {actions['mark_as_task']['task_due_date']}")
                            if 'play_sound' in actions and actions['play_sound']['sound_file']:
                                import winsound
                                winsound.PlaySound(actions['play_sound']['sound_file'], winsound.SND_FILENAME)
                                self.debug_print(f"Played sound: {actions['play_sound']['sound_file']}")
                            if 'display_desktop_alert' in actions and actions['display_desktop_alert']:
                                self.debug_print("Desktop alert displayed")
                                # Implement desktop alert display logic here
                            if 'copy_to_folder' in actions and actions['copy_to_folder']['folder_name']:
                                folder_name = actions['copy_to_folder']['folder_name']
                                target_folder = self.target_folder.Folders[folder_name]
                                email.Copy().Move(target_folder)
                                self.debug_print(f"Email copied to '{folder_name}' folder")
                            if 'forward' in actions and actions['forward']:
                                forward_recipients = [recipient['address'] for recipient in actions['forward']]
                                forward_email = email.Forward()
                                forward_email.To = ";".join(forward_recipients)
                                forward_email.Send()
                                self.debug_print(f"Email forwarded to: {', '.join(forward_recipients)}")
                            if 'reply' in actions and actions['reply']['template']:
                                reply_email = email.Reply()
                                reply_email.Body = actions['reply']['template']
                                reply_email.Send()
                                self.debug_print("Auto-reply sent")
                            if 'redirect' in actions and actions['redirect']:
                                redirect_recipients = [recipient['address'] for recipient in actions['redirect']]
                                redirect_email = email.Forward()
                                redirect_email.To = ";".join(redirect_recipients)
                                redirect_email.Send()
                                self.debug_print(f"Email redirected to: {', '.join(redirect_recipients)}")
                            if 'print' in actions and actions['print']:
                                email.PrintOut()
                                self.debug_print("Email printed")
                            if 'run_script' in actions and actions['run_script']['script_path']:
                                exec(open(actions['run_script']['script_path']).read())
                                self.debug_print(f"Script executed: {actions['run_script']['script_path']}")
                            if 'start_application' in actions and actions['start_application']['application_path']:
                                import subprocess
                                subprocess.Popen(actions['start_application']['application_path'])
                                self.debug_print(f"Application started: {actions['start_application']['application_path']}")
                            if 'move_to_folder' in actions and actions['move_to_folder']['folder_name']:
                                folder_name = actions['move_to_folder']['folder_name']
                                target_folder = self.target_folder.Folders[folder_name]
                                email.Move(target_folder)
                                self.debug_print(f"Email moved to '{folder_name}' folder")
                            if 'stop_processing_more_rules' in actions and actions['stop_processing_more_rules']:
                                self.debug_print("Stopping processing more rules")
                                # this flag is not being passed by outlook, so will never be set.  Keeping in case fixed in the future
                            if 'delete' in actions and actions['delete']:
                                if email.UnRead:
                                    email.UnRead = False  # Delete implies marking the item as read
                                    self.debug_print(f"Email marked as read")
                                if hasattr(email, 'Flag'):
                                    email.Flag.Clear()      # Delete implies clearing the flag
                                    self.debug_print(f"Email flag was cleared")
                                # assign category based on rule name
                                try:
                                    rule_name = rule['name']
                                    category_name = self.rule_to_category.get(rule_name, actions['assign_to_category']['category_name'])
                                    email.Categories = category_name
                                    email.Save()
                                    self.debug_print(f"Email assigned to category '{category_name}'")
                                except Exception as e:
                                    self.debug_print(f"Error assigning category to email: {str(e)}")

                                try:
                                    # delete email
                                    email.Delete()
                                    email_deleted = True
                                    self.debug_print("Email: marked as read, flag cleared and deleted")
                                    # delete implies "Stop Processing More Rules".  Continue will go to next email
                                except Exception as e:
                                    self.debug_print(f"Error deleting email: {str(e)}")

                                break # If delete, then process no more rules and go to next email
                            continue  # Go to the next email if one rule matches

                    # After all email rules are processed and it did not match any rules and the email has not been deleted, then check for phishing indicators
                    if not (email_deleted):
                        indicators = self.check_phishing_indicators(email)
                        if indicators:
                            flagged_count += 1
                            self.debug_print(f"Phishing indicators found: {indicators}")
                        else:
                            self.debug_print("No conditions or phishing indicators found")
                        # If it is in the Bulk Mail folder, but nothing indicated via rules or phishing,
                        # show the body and header, so we information needed to add it to a rule
                        for line in email.Body.splitlines():
                            self.debug_print(f"Body: {line}")
                        for header in email_header.splitlines():
                            self.debug_print(f"Header: {header}")


                except Exception as e:
                    self.debug_print(f"Error processing email: {str(e)}")

            self.debug_print(f"\nProcessing Summary:")
            self.debug_print(f"Processed {processed_count} emails")
            self.debug_print(f"Flagged {flagged_count} emails as suspicious")

        except Exception as e:
            self.debug_print(f"Error in process_emails: {str(e)}")
            raise

In [ ]:
"""Main function to run the security agent"""
try:
    print("\nStarting Outlook Security Agent with DEBUG = {DEBUG}")
    print("This will make changes")
    print("Check the console output and outlook_security.log for detailed information")

    # Initialize agent with debug mode enabled
    agent = OutlookSecurityAgent()  # call with defaults
    rules_json = agent.get_outlook_rules()
    #print(rules_json)

    # Process last 24 hours of emails
    agent.process_emails(rules_json)

    print("Complete. Check the log file for detailed analysis.\n\n")

except Exception as e:
    print(f"\nError: {str(e)}")
    logging.error(f"Main execution error: {str(e)}")